In [36]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess

import os
import time
import datetime
import calendar
from typing import Tuple

import geemap

ee.Authenticate()
ee.Initialize(project='ee-juliocesarborgesdeoliv-neve')

data = {}

In [63]:
def calcNDSI(img):
    # print("bands=",img.bandNames().getInfo())
    # ndsi = img.select('GREEN').subtract(img.select('INFRARED')).divide(img.select('GREEN').add(img.select('INFRARED'))).rename('NDSI')
    ndsi = img.normalizedDifference(['GREEN','INFRARED']).rename('NDSI')
    mask = img.expression('(1 != (b("QA_PIXEL")==22280)+(b("QA_PIXEL")==24088)+(b("QA_PIXEL")==24216)+(b("QA_PIXEL")==24344)+(b("QA_PIXEL")==24472)+(b("QA_PIXEL")==55052))').rename('MASK')
    ret = ndsi.updateMask(mask)
    return ret, mask, ndsi

def get_landsat_images(min_date: datetime.date, max_date: datetime.date, study_area_broad) -> ee.ImageCollection:
  min_date_str = min_date.strftime("%Y-%m-%d")
  max_date_str = max_date.strftime("%Y-%m-%d")
  # print(min_date_str); print(max_date_str);

  landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(study_area_broad).sort("CLOUD_COVER")
  landsat_8 = landsat_8.select(['SR_B3', 'SR_B6', 'QA_PIXEL'],['GREEN', 'INFRARED', 'QA_PIXEL'])

  landsat_9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterBounds(study_area_broad).sort("CLOUD_COVER")
  landsat_9 = landsat_9.select(['SR_B3', 'SR_B6', 'QA_PIXEL'],['GREEN', 'INFRARED', 'QA_PIXEL'])

  landsat_7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterBounds(study_area_broad).sort("CLOUD_COVER")
  landsat_7 = landsat_7.select(['SR_B2', 'SR_B5', 'QA_PIXEL'],['GREEN', 'INFRARED', 'QA_PIXEL'])

  landsat_5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterBounds(study_area_broad).sort("CLOUD_COVER")
  landsat_5 = landsat_5.select(['SR_B2', 'SR_B5', 'QA_PIXEL'],['GREEN', 'INFRARED', 'QA_PIXEL'])

  landsat_4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2").filterBounds(study_area_broad).sort("CLOUD_COVER")
  landsat_4 = landsat_4.select(['SR_B2', 'SR_B5', 'QA_PIXEL'],['GREEN', 'INFRARED', 'QA_PIXEL'])

  all_landsat = landsat_8.merge(landsat_9).merge(landsat_5).merge(landsat_4).merge(landsat_7).filterDate(min_date_str,max_date_str)
  all_landsat = all_landsat.filter(ee.Filter.calendarRange(6,18,'hour'))
  return all_landsat


def export_image(img, name, folder, study_area, description='NDSI'):
  print("saving",name)
  projection = img.projection().getInfo()
  task = ee.batch.Export.image.toDrive(
    image=img.clip(study_area),
    description=description,
    folder=folder,
    fileNamePrefix=name,
    region=study_area.geometry(),
    scale=30,
    crs='EPSG:32618'
    )
  task.start()

def gen_NDSIs(min_date:datetime.date, max_date:datetime.date, export = True, limit = None, list_images = True) -> list:
  imgs_range = []
  str_range: str = str(min_date) + " -> " + str(max_date)
  print("Range (" + str_range + ")")
  images: ee.ImageCollection = get_landsat_images(min_date, max_date,study_area_broad)
  if limit != None: images = images.limit(limit)
  img_list = images.toList(len(images.getInfo()['features']))
  count = len(images.getInfo()['features'])
  print("found",count,"images")
  if count == 0: return None
  info = images.getInfo()
  for i in range(0,count):
    if list_images: print("Found",images.getInfo()['features'][i]['id'])
    img = ee.Image(img_list.get(i)).clip(study_area_broad)
    img_ndsi, mask, original = calcNDSI(img)
    imgs_range.append(img_ndsi)
  imgs_range = ee.ImageCollection(imgs_range)
  local_mean = imgs_range.mean()
  filename = "NDSI_"+str_range
  data[filename] = imgs_range.mean()
  if export: export_image(local_mean, filename, "NDSI_IMGS", study_area_broad, description='NDSI_'+str_range.replace('->','to'))
  return imgs_range

def gen_coverage(min_date:datetime.date, max_date:datetime.date, export = True, limit = None, threashold = 0.4):
  imgs_range = gen_NDSIs(min_date, max_date, export=False, limit=limit)
  if imgs_range == None: return None
  img = imgs_range.mean().gt(threashold)
  str_range: str = str(min_date) + " -> " + str(max_date)
  filename = "NDSI_CVRG_"+str_range
  if export: export_image(img, filename, "NDSI_CVRG", study_area_broad, description='NDSI_CVRG_'+str_range.replace('->','to'))
  data[filename] = img
  return img


In [ ]:
print("Olá mundo")
# TODO salva tudo num dict
study_area_broad = ee.FeatureCollection('projects/ee-juliocesarborgesdeoliv-neve/assets/SanRafael')
i = 0

export_year, export_station = True, False

for year in range(2002, 2003):
  print("\nYear {:04d}:\n".format(year))
  if export_year:
    print("TOTAL:")
    local_mean = gen_NDSIs(datetime.date(year,1,1),datetime.date(year,12,31)).mean()
  if export_station:
    print("STATIONS:")
    print("SUMMER:")
    gen_NDSIs(datetime.date(year-1,12,21),datetime.date(year,3,20))
    print("FALL:")
    gen_NDSIs(datetime.date(year,3,21),datetime.date(year,6,20))
    print("WINTER:")
    gen_NDSIs(datetime.date(year,6,21),datetime.date(year,9,22))
    print("SPRING:")
    gen_NDSIs(datetime.date(year,9,23),datetime.date(year,12,20))

In [68]:
print("Olá mundo 2")
# TODO salva tudo num dict
study_area_broad = ee.FeatureCollection('projects/ee-juliocesarborgesdeoliv-neve/assets/SanRafael')
i = 0

export_year, export_station = True, False

for year in range(1999, 2024):
  print("\nYear {:04d}:\n".format(year))
  if export_year:
    print("TOTAL:")
    gen_coverage(datetime.date(year,1,1),datetime.date(year,12,31),limit=40)
  if export_station:
    print("STATIONS:")
    print("SUMMER:")
    gen_coverage(datetime.date(year-1,12,21),datetime.date(year,3,20))
    print("FALL:")
    gen_coverage(datetime.date(year,3,21),datetime.date(year,6,20))
    print("WINTER:")
    gen_coverage(datetime.date(year,6,21),datetime.date(year,9,22))
    print("SPRING:")
    gen_coverage(datetime.date(year,9,23),datetime.date(year,12,20))


Olá mundo 2

Year 1999:

TOTAL:
Range (1999-01-01 -> 1999-12-31)
1999-01-01
1999-12-31
found 40 images
Found LANDSAT/LT05/C02/T1_L2/LT05_001092_19990312
Found LANDSAT/LT05/C02/T1_L2/LT05_231092_19990408
Found LANDSAT/LT05/C02/T1_L2/LT05_231092_19990307
Found LANDSAT/LT05/C02/T1_L2/LT05_232093_19990821
Found LANDSAT/LT05/C02/T1_L2/LT05_233093_19991031
Found LANDSAT/LT05/C02/T1_L2/LT05_232093_19991008
Found LANDSAT/LT05/C02/T1_L2/LT05_232092_19990821
Found LANDSAT/LT05/C02/T1_L2/LT05_233093_19990812
Found LANDSAT/LT05/C02/T1_L2/LT05_233092_19990812
Found LANDSAT/LT05/C02/T1_L2/LT05_232092_19990210
Found LANDSAT/LT05/C02/T1_L2/LT05_232092_19990125
Found LANDSAT/LT05/C02/T1_L2/LT05_232093_19990210
Found LANDSAT/LT05/C02/T1_L2/LT05_231093_19990307
Found LANDSAT/LT05/C02/T1_L2/LT05_231092_19990915
Found LANDSAT/LT05/C02/T1_L2/LT05_232092_19991008
Found LANDSAT/LT05/C02/T1_L2/LT05_231093_19991001
Found LANDSAT/LT05/C02/T1_L2/LT05_001093_19990312
Found LANDSAT/LT05/C02/T1_L2/LT05_232092_199901

In [5]:
study_area_broad = ee.FeatureCollection('projects/ee-juliocesarborgesdeoliv-neve/assets/SanRafael')
gen_coverage(datetime.date(1998,1,1),datetime.date(2005,12,31))
gen_coverage(datetime.date(2006,1,1),datetime.date(2014,12,31))
gen_coverage(datetime.date(2015,1,1),datetime.date(2023,12,31))

In [59]:
print('DATA:')
for i in data: print(i)

DATA:
NDSI_1998-01-01 -> 1998-12-31
NDSI_CVRG_1998-01-01 -> 1998-12-31
NDSI_1999-01-01 -> 1999-12-31
NDSI_CVRG_1999-01-01 -> 1999-12-31
NDSI_2000-01-01 -> 2000-12-31
NDSI_CVRG_2000-01-01 -> 2000-12-31
NDSI_2001-01-01 -> 2001-12-31
NDSI_CVRG_2001-01-01 -> 2001-12-31
NDSI_2002-01-01 -> 2002-12-31
NDSI_CVRG_2002-01-01 -> 2002-12-31
NDSI_2003-01-01 -> 2003-12-31
NDSI_CVRG_2003-01-01 -> 2003-12-31
NDSI_2004-01-01 -> 2004-12-31
NDSI_CVRG_2004-01-01 -> 2004-12-31
NDSI_2005-01-01 -> 2005-12-31
NDSI_CVRG_2005-01-01 -> 2005-12-31
NDSI_2006-01-01 -> 2006-12-31
NDSI_CVRG_2006-01-01 -> 2006-12-31
NDSI_2007-01-01 -> 2007-12-31
NDSI_CVRG_2007-01-01 -> 2007-12-31
NDSI_2008-01-01 -> 2008-12-31
NDSI_CVRG_2008-01-01 -> 2008-12-31
NDSI_2009-01-01 -> 2009-12-31
NDSI_CVRG_2009-01-01 -> 2009-12-31
NDSI_2010-01-01 -> 2010-12-31
NDSI_CVRG_2010-01-01 -> 2010-12-31
NDSI_2011-01-01 -> 2011-12-31
NDSI_CVRG_2011-01-01 -> 2011-12-31
NDSI_2012-01-01 -> 2012-12-31
NDSI_CVRG_2012-01-01 -> 2012-12-31
NDSI_2013-01-01 -> 

In [ ]:
gen_coverage(datetime.date(2000,1,1),datetime.date(2000,12,31),export = False)
gen_coverage(datetime.date(2005,1,1),datetime.date(2005,12,31),export = False)
gen_coverage(datetime.date(2010,1,1),datetime.date(2010,12,31),export = False)
gen_coverage(datetime.date(2020,1,1),datetime.date(2020,12,31),export = False)


In [76]:
map = geemap.Map(center=[-46.5330, -75.0132], zoom=8)

map.add_layer(study_area_broad, {}, 'area')

def show_year(year: int, NDSI = True, COVERAGE = True):
  if NDSI: map.add_layer(data['NDSI_{:04d}-01-01 -> {:04d}-12-31'.format(year,year)], {min: -0, max: .5}, 'NDSI {:04d}'.format(year))
  if COVERAGE: map.add_layer(data['NDSI_CVRG_{:04d}-01-01 -> {:04d}-12-31'.format(year,year)], {}, 'SNOW {:04d}'.format(year))

# for i in range(1998,2023,3):
#   show_year(i,NDSI=False)
show_year(1999,NDSI=True,COVERAGE=False)
show_year(2001,NDSI=True,COVERAGE=False)
show_year(2005,NDSI=True,COVERAGE=False)
show_year(2010,NDSI=True,COVERAGE=False)
show_year(2015,NDSI=True,COVERAGE=False)
show_year(2020,NDSI=True,COVERAGE=False)

display(map)
print(data)

Map(center=[-46.533, -75.0132], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

{'NDSI_1998-01-01 -> 1998-12-31': <ee.image.Image object at 0x7be3a6008850>, 'NDSI_CVRG_1998-01-01 -> 1998-12-31': <ee.image.Image object at 0x7be3a600b1f0>, 'NDSI_1999-01-01 -> 1999-12-31': <ee.image.Image object at 0x7be3a5c90640>, 'NDSI_CVRG_1999-01-01 -> 1999-12-31': <ee.image.Image object at 0x7be3a5c91f00>, 'NDSI_2000-01-01 -> 2000-12-31': <ee.image.Image object at 0x7be3ab6c45e0>, 'NDSI_CVRG_2000-01-01 -> 2000-12-31': <ee.image.Image object at 0x7be3ab6c70d0>, 'NDSI_2001-01-01 -> 2001-12-31': <ee.image.Image object at 0x7be3abd09300>, 'NDSI_CVRG_2001-01-01 -> 2001-12-31': <ee.image.Image object at 0x7be3abd095d0>, 'NDSI_2002-01-01 -> 2002-12-31': <ee.image.Image object at 0x7be3abad23b0>, 'NDSI_CVRG_2002-01-01 -> 2002-12-31': <ee.image.Image object at 0x7be3abad2890>, 'NDSI_2003-01-01 -> 2003-12-31': <ee.image.Image object at 0x7be3ab3d0c10>, 'NDSI_CVRG_2003-01-01 -> 2003-12-31': <ee.image.Image object at 0x7be3ab3d2080>, 'NDSI_2004-01-01 -> 2004-12-31': <ee.image.Image object a

In [60]:
# Para a máquina não desligar e descartar as variáveis de ambiente
it = 0
while True:
  print('zzz',it)
  time.sleep(10)
  it += 1

zzz 0
zzz 1
zzz 2
zzz 3
zzz 4
zzz 5
zzz 6
zzz 7
zzz 8
zzz 9
zzz 10
zzz 11
zzz 12
zzz 13
zzz 14
zzz 15
zzz 16
zzz 17
zzz 18
zzz 19
zzz 20
zzz 21
zzz 22
zzz 23
zzz 24
zzz 25
zzz 26
zzz 27
zzz 28
zzz 29
zzz 30
zzz 31
zzz 32
zzz 33
zzz 34
zzz 35
zzz 36
zzz 37
zzz 38
zzz 39
zzz 40
zzz 41
zzz 42
zzz 43
zzz 44
zzz 45
zzz 46
zzz 47
zzz 48
zzz 49
zzz 50
zzz 51
zzz 52
zzz 53
zzz 54
zzz 55
zzz 56
zzz 57
zzz 58
zzz 59
zzz 60
zzz 61
zzz 62
zzz 63
zzz 64
zzz 65
zzz 66
zzz 67
zzz 68
zzz 69
zzz 70
zzz 71
zzz 72
zzz 73
zzz 74
zzz 75
zzz 76
zzz 77
zzz 78
zzz 79
zzz 80
zzz 81
zzz 82
zzz 83
zzz 84
zzz 85
zzz 86
zzz 87
zzz 88
zzz 89
zzz 90
zzz 91
zzz 92
zzz 93
zzz 94
zzz 95
zzz 96
zzz 97
zzz 98
zzz 99
zzz 100
zzz 101
zzz 102
zzz 103
zzz 104
zzz 105
zzz 106
zzz 107
zzz 108
zzz 109
zzz 110
zzz 111
zzz 112
zzz 113
zzz 114
zzz 115
zzz 116
zzz 117
zzz 118
zzz 119
zzz 120
zzz 121
zzz 122
zzz 123
zzz 124
zzz 125
zzz 126
zzz 127
zzz 128
zzz 129
zzz 130
zzz 131
zzz 132
zzz 133
zzz 134
zzz 135
zzz 136
zzz 137
zzz 13

KeyboardInterrupt: 